In [1]:
# make a pkl file of adj matrix between GFC2 and GFC4 preMNs and mn ids from v604 and the muscle they innervate so that the figure 6 notebook doesnt need a cave token

In [1]:
# import packages
import pandas as pd
import numpy as np
from matplotlib import pyplot,patches
import matplotlib.pyplot as plt
import seaborn as sns
from caveclient import CAVEclient
import os

client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)

In [2]:
# get left wing MNs from cave annotation table
wing_table = client.materialize.query_table('wing_motor_neuron_table_v0')

wing_table['side'] = [x.strip()[-1] for x in wing_table.classification_system]
wingL_table = wing_table[wing_table.side.isin(['L'])]

wing_MN_table = wingL_table[~wingL_table.cell_type.isin(['nonMN_unk3','nonMN_unk2','nonMN_unk1','nonMN_PSI','tt','ttb_u','ttc_u', 'PSn_u'])] # other efferent neurons
wing_MN_table.shape # should be 29

(29, 10)

In [3]:
# get left leg MNs from cave annotation table
leg_table = client.materialize.query_table('motor_neuron_table_v7')

leg_table['side'] = [x.strip()[-1] for x in leg_table.classification_system]
leg_MN_table = leg_table[leg_table.side.isin(['L'])]

leg_MN_table.shape # should be 69

(69, 10)

In [4]:
mn_df = pd.concat([wing_MN_table,leg_MN_table])

In [5]:
# query for the full connectivity matrix
full_df = client.materialize.synapse_query(post_ids = mn_df.pt_root_id.to_list())

In [6]:
# unique IDs for preMNs, constrain presynaptic partners to preMNs of interest (electrically coupled to giant fiber)

escape_preMNs = [648518346514282206,648518346494160395,648518346489931738,648518346494622635,648518346495640464,
                 648518346479395264,648518346483242404,648518346526182103,648518346472208434,648518346483122340,648518346486576431]

In [7]:
full_escape_df = full_df[full_df.pre_pt_root_id.isin(escape_preMNs)]

print(len(full_escape_df.pre_pt_root_id.value_counts())) # should be 11
print(len(escape_preMNs)) # should be 11

print(len(full_df.post_pt_root_id.value_counts())) # should be 98

11
11
98


In [8]:
# only include >3 synapse strength connections
def group_and_count_inputs(df, thresh):

    # count the number of synapses between pairs of pre and post synaptic inputs
    syn_in_conn=df.groupby(['pre_pt_root_id','post_pt_root_id']).transform(len)['id']
    # save this result in a new column and reorder the index
    df['syn_in_conn']=syn_in_conn
    df = df[['id', 'pre_pt_root_id','post_pt_root_id','score','syn_in_conn']].sort_values('syn_in_conn', ascending=False).reset_index()

    # Filter out small synapses between pairs of neurons and now print the shape
    df = df[df['syn_in_conn']>=thresh]
    # print(df.shape)
    return df

In [9]:
print(len(full_escape_df.post_pt_root_id.value_counts()))

escape_df = group_and_count_inputs(full_escape_df, 3)

print(len(escape_df.post_pt_root_id.value_counts()))


43
26


/var/folders/kc/3ksx1s8x04xcd7r2v8zsqqtm0000gn/T/ipykernel_1051/418363652.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['syn_in_conn']=syn_in_conn


In [10]:
# include synapse coordinates
escape_df = escape_df.merge(full_escape_df[['id','pre_pt_position','post_pt_position']])

In [19]:
# label pre_id as GFC4 or GFC2 and post id by mn label
mn_df['post_pt_root_id'] = mn_df['pt_root_id']
GFC4 = [648518346483242404,648518346472208434,648518346479395264,648518346526182103,648518346495640464,648518346483122340,648518346494622635,648518346486576431]
GFC2 = [648518346514282206,648518346489931738,648518346494160395]

escape_df = escape_df.merge(mn_df[['post_pt_root_id','cell_type']])
escape_df.loc[escape_df['pre_pt_root_id'].isin(GFC4), 'preMN_type'] = 'GFC4'
escape_df.loc[escape_df['pre_pt_root_id'].isin(GFC2), 'preMN_type'] = 'GFC2'
escape_df.preMN_type.value_counts().sum() #should add up to 1150

1150

In [21]:
escape_df.to_pickle('escape_df.pkl')